Author: Adafaly Matthieu </br>

This notebook still has some modifications to be made and is not finished yet.

# Importing libraries


In [ ]:
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import shapiro
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import calendar
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Data


In [ ]:
df = pd.read_pickle("Data/pollution_rennes_filter.pkl")
print("dataframe loaded")

In [ ]:
display(df)

In [ ]:
df_stationnary = df.loc[(df['sensor_type'] == 'fixedGps') & (df['PM_2.5'].notna())]
df_mobile = df.loc[(df['sensor_type'] == 'mobileGps') & (df['PM_2.5'].notna())]

## Data visualization

In [ ]:
# Calculate the hourly average PM₂.₅ per sensor
mean_values = df.groupby(['hour', 'sensor_name'])['PM_2.5'].mean().reset_index()

# Remove a specific station if needed
mean_values = mean_values[mean_values['sensor_name'] != 'standalone-LOPY-AQ05']

# Set the figure size
plt.figure(figsize=(12, 6))

# Plot the lines using Seaborn
sns.lineplot(data=mean_values, x='hour', y='PM_2.5', hue='sensor_name')

# Add title and labels
plt.title("Average PM₂.₅ Concentration by Hour and Sensor")
plt.xlabel("Time of Day (Hour)")
plt.ylabel("PM₂.₅ Concentration (µg/m³)")
plt.legend(title="Station", bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Define the order of the days of the week
week_days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['day_week'] = pd.Categorical(df['day_week'], categories=week_days_order, ordered=True)

# Compute the average PM2.5 concentration
mean_values = df.groupby(['day_week', 'sensor_name'], observed=True)['PM_2.5'].mean().reset_index()

# Create the line plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=mean_values, x='day_week', y='PM_2.5', hue='sensor_name', marker='o')

# Add title and axis labels
plt.title("Average PM₂.₅ Concentration by Day of the Week and Sensor")
plt.xlabel("Day of the Week")
plt.ylabel("PM₂.₅ Concentration (µg/m³)")

# Show legend on the right
plt.legend(title="Sensor", bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Create a boxplot with Plotly
df_box = df.copy()
df_box['sensor_name'] = df_box.index.get_level_values('sensor_name')

fig = px.box(df_box, 
             x="sensor_name", 
             y="PM_2.5", 
             title="Distribution of pollution value by sensor", 
             labels={"PM_2.5": "Pollution value (µg/m³)", "sensor_name": "Sensor name"})

# Update the margins to recenter the box
fig.update_layout(
    margin=dict(l=40, r=40, t=40, b=40),  # Adjust the left, right, top, bottom margins
    boxmode='group',  # Ensure that the boxes do not overlap
    yaxis=dict(
        range=[df['PM_2.5'].quantile(0.05), df['PM_2.5'].quantile(0.95)]  # Limit the y-axis to the 5-95% of the data
    )
)
# Display the graph
fig.show()

In [ ]:
lissage=14
global_mean = df.groupby(
    df.index.get_level_values('measure_date').floor('D')
)['PM_2.5'].mean().reset_index()

global_mean.columns = ['date', 'mean_v']


# Adding the smoothed mean
global_mean['smoothed_mean'] = global_mean['mean_v'].rolling(window=lissage, center=True).mean()

# Interactive plot of the global mean
fig = px.line(
    global_mean,
    x='date',
    y='mean_v',
    markers=True,
    title="Daily Mean PM₂.₅ Concentration",
    labels={
        'mean_v': 'Concentration (µg/m³)',
        'date': 'Date'
    }
)

# Adding the smoothed line (black line)
fig.add_scatter(
    x=global_mean['date'],
    y=global_mean['smoothed_mean'],
    mode='lines',
    name=f'Smoothed mean ({lissage} days)',
    line=dict(color='black', width=3)
)
fig.update_traces(line_color='darkorange', line_width=3, selector=dict(name=None))  # main line
fig.update_layout(hovermode='x unified')

fig.show()

### PM₂.₅ Data Distribution Across Different Variables


In [ ]:
# Extract all PM₂.₅ values (excluding missing values)
data = df['PM_2.5'].dropna()

# Create the figure
plt.figure(figsize=(10, 6))

# Histogram 
sns.histplot(data, bins=200, kde=True, color='orange', stat="density", edgecolor=None, label='Actual data')

# Theoretical normal distribution curve
mu = data.mean()
sigma = data.std()
x = np.linspace(0, 40, 500)
y = norm.pdf(x, mu, sigma)
plt.plot(x, y, color='red', linestyle='--', linewidth=2, label=f'Normal distribution\nμ={mu:.2f}, σ={sigma:.2f}')

# Graph adjustments
plt.title("Overall PM₂.₅ Distribution Normal Curve")
plt.xlabel("PM₂.₅ (µg/m³)")
plt.ylabel("Density")
plt.xlim(0, 40)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
data = df['PM_2.5'].dropna()
# Histogram 
sns.histplot(data, bins=200, kde=True, color='orange', stat="density", edgecolor=None, label='Actual data')

# Theoretical normal distribution curve
mu = data.mean()
sigma = data.std()
x = np.linspace(0, 40, 500)
y = norm.pdf(x, mu, sigma)
plt.plot(x, y, color='red', linestyle='--', linewidth=2, label=f'Normal distribution\nμ={mu:.2f}, σ={sigma:.2f}')

# Graph adjustments
plt.title("Overall PM₂.₅ Distribution Normal Curve")
plt.xlabel("PM₂.₅ (µg/m³)")
plt.ylabel("Density")
plt.xlim(0, 40)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
df['PM_2.5'] = pd.to_numeric(df['PM_2.5'], errors='coerce')

In [ ]:
data = df['PM_2.5']
stats.probplot(data, dist="norm", plot=plt)
plt.title("Q-Q plot for log(PM2.5)")
plt.grid()
plt.show()


### Aggregate by day

In [ ]:
df_stationnary = df.loc[(df['sensor_type'] == 'fixedGps') & (df['PM_2.5'].notna())]

In [ ]:
df_day = df.groupby(
    df.index.get_level_values('measure_date').floor('D')
)['PM_2.5'].mean().reset_index()
df_day['measure_date'] = df_day['measure_date'].dt.tz_localize(None)  
display(df_day)

In [ ]:
df_day['year_month'] = df_day['measure_date'].dt.to_period('M')  # Format AAAA-MM
count_per_month = df_day['year_month'].value_counts().sort_index()
print(count_per_month)

We use the Shapiro test to check whether the data distribution follows a normal distribution.
To do this, we use the null hypothesis (H₀): the distribution is assumed to be normal.
We performed the test on the original data and then on the data after applying the logarithmic function.

In [ ]:
data = df_day['PM_2.5']   
stat, p_value = shapiro(data)
print(f"Shapiro-Wilk Test: statistic = {stat:.4f}, p-value = {p_value:.4f}")
if p_value > 0.05:
    print("✅ The data probably follows a normal distribution.")
else:    
    print("❌ The data does not follow a normal distribution.")


In [ ]:
data = np.log(df_day['PM_2.5'] + 1) 
stat, p_value = shapiro(data)
print(f"Shapiro-Wilk Test: statistic = {stat:.4f}, p-value = {p_value:.4f}")
if p_value > 0.05:
    print("✅ The data probably follows a normal distribution.")
else:    print("❌ The data does not follow a normal distribution.")


In [ ]:
fig = px.line(
    df_day,
    x='measure_date',
    y='PM_2.5',
    markers=True,
    title="Daily Mean PM₂.₅ Concentration",
    labels={
        'mean_v': 'Concentration (µg/m³)',
        'date': 'Date'
    }
)
fig.show()

In [ ]:
serie = df_day['PM_2.5']
# Apply the lbjung-box test
result = acorr_ljungbox(serie, lags=[100])
print(result)

In [ ]:
# Create a figure with 2 subplots
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# Plot autocorrelation (ACF)
plot_acf(serie, ax=axes[0], lags=100)
axes[0].set_title('Autocorrelation (ACF)')

# Plot partial autocorrelation (PACF)
plot_pacf(serie, ax=axes[1], lags=100)
axes[1].set_title('Partial Autocorrelation (PACF)')

plt.show()


In [ ]:
decomposition = seasonal_decompose(df_day['PM_2.5'], model='additive', period=7)

trend = decomposition.trend
seasonal = decomposition.seasonal
resid = decomposition.resid
fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

# Original series
axs[0].plot(df_day['PM_2.5'], label='Original')
axs[0].legend(loc='upper left')
axs[0].set_title('Original Series')

# Trend
axs[1].plot(trend, label='Trend', color='orange')
axs[1].legend(loc='upper left')
axs[1].set_title('Trend')

# Seasonal
axs[2].plot(seasonal, label='Seasonality', color='green')
axs[2].legend(loc='upper left')
axs[2].set_title('Seasonality')

# Residuals
axs[3].plot(resid, label='Residuals', color='red')
axs[3].legend(loc='upper left')
axs[3].set_title('Residuals')

plt.tight_layout()
plt.show()


In [ ]:
# Decompositions with different periods
decomp_7 = seasonal_decompose(df_day['PM_2.5'], model='additive', period=7)
decomp_30 = seasonal_decompose(df_day['PM_2.5'], model='additive', period=30)

# Create the figure
fig, axs = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Seasonal component with period = 7
axs[0].plot(decomp_7.seasonal, label='Seasonality (P=7)', color='green')
axs[0].legend(loc='upper left')
axs[0].set_title('Seasonality (period = 7 days)')

# Seasonal component with period = 30
axs[1].plot(decomp_30.seasonal, label='Seasonality (P=30)', color='blue')
axs[1].legend(loc='upper left')
axs[1].set_title('Seasonality (period = 30 days)')

plt.tight_layout()
plt.show()


In [ ]:
print("Mean of residuals:", resid.mean())


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(resid.dropna(), lags=30)
plt.title("ACF of residuals")
plt.show()


We study the stationarity of a time series to determine whether its statistical properties — such as mean, variance, and autocorrelation — remain constant over time or change.

In [ ]:
result = adfuller(decomposition.resid.dropna())
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
mean_day = df.groupby('day_week',observed=False)['PM_2.5'].mean()
mean_day = mean_day.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
print(mean_day)

In [ ]:
plt.figure(figsize=(8,5))
mean_day.plot(kind='bar', color='skyblue')
plt.title('Mean of the pollution PM2.5 by day of the week')
plt.ylabel('PM2.5 moyen')
plt.xlabel('Jour de la semaine')
plt.show()


In [ ]:
mean_month = df.groupby('month')['PM_2.5'].mean()

plt.figure(figsize=(8,5))
mean_month.plot(kind='bar', color='coral')
plt.title('mean of pollution PM2.5 by month')
plt.ylabel('mean_PM2.5')
plt.xlabel('Month')
plt.show()
